In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anuvarshini98","key":"0c1621206d15c49de511192782db8fcc"}'}

In [2]:
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [3]:
!pip install --upgrade --force-reinstall --no-deps kaggle

Processing /root/.cache/pip/wheels/3a/d1/7e/6ce09b72b770149802c653a02783821629146983ee5a360f10/kaggle-1.5.10-cp36-none-any.whl
  Found existing installation: kaggle 1.5.10
    Uninstalling kaggle-1.5.10:
      Successfully uninstalled kaggle-1.5.10


In [4]:
!kaggle competitions download -c tabular-playground-series-jan-2021

tabular-playground-series-jan-2021.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
with zipfile.ZipFile('/content/tabular-playground-series-jan-2021.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [6]:
import pandas as pd
import numpy as np


In [7]:
train = pd.read_csv("/content/train.csv")
test = pd.read_csv("/content/test.csv")

In [8]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

from tqdm import tqdm
import xgboost as xgb

In [9]:
features = ['cont1', 'cont2', 'cont3', 'cont4','cont5','cont6', 'cont7',
       'cont8', 'cont9', 'cont10', 'cont11', 'cont12', 'cont13','cont14']

In [10]:
X_train = train[features]
y_train = train["target"]
final_X_test = test[features]

In [11]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.1, random_state=0)

In [12]:
regressor = xgb.XGBRegressor(
                 colsample_bytree=0.5,
                 alpha=0.01563,
                 #gamma=0.0,
                 learning_rate=0.01,
                 max_depth=15,
                 min_child_weight=257,
                 n_estimators=4000,                                                                  
                 #reg_alpha=0.9,
                 reg_lambda=0.003,
                 subsample=0.7,
                 random_state=2020,
                 metric_period=100,
                 silent=1)

In [13]:
regressor.fit(X_train, y_train, early_stopping_rounds=10, eval_set=[(X_test, y_test)], verbose=1)

[0]	validation_0-rmse:7.37015
Will train until validation_0-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:7.29719
[2]	validation_0-rmse:7.22493
[3]	validation_0-rmse:7.15338
[4]	validation_0-rmse:7.08261
[5]	validation_0-rmse:7.01252
[6]	validation_0-rmse:6.9432
[7]	validation_0-rmse:6.87449
[8]	validation_0-rmse:6.80657
[9]	validation_0-rmse:6.73927
[10]	validation_0-rmse:6.67268
[11]	validation_0-rmse:6.60675
[12]	validation_0-rmse:6.54149
[13]	validation_0-rmse:6.47688
[14]	validation_0-rmse:6.41296
[15]	validation_0-rmse:6.34965
[16]	validation_0-rmse:6.28697
[17]	validation_0-rmse:6.22492
[18]	validation_0-rmse:6.16351
[19]	validation_0-rmse:6.10272
[20]	validation_0-rmse:6.04256
[21]	validation_0-rmse:5.98302
[22]	validation_0-rmse:5.92406
[23]	validation_0-rmse:5.86573
[24]	validation_0-rmse:5.80798
[25]	validation_0-rmse:5.75081
[26]	validation_0-rmse:5.69423
[27]	validation_0-rmse:5.63822
[28]	validation_0-rmse:5.58277
[29]	validation_0-rmse:5.5279
[30]	validation_0

XGBRegressor(alpha=0.01563, base_score=0.5, booster='gbtree',
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.5,
             gamma=0, importance_type='gain', learning_rate=0.01,
             max_delta_step=0, max_depth=15, metric_period=100,
             min_child_weight=257, missing=None, n_estimators=4000, n_jobs=1,
             nthread=None, objective='reg:linear', random_state=2020,
             reg_alpha=0, reg_lambda=0.003, scale_pos_weight=1, seed=None,
             silent=1, subsample=0.7, verbosity=1)

In [14]:
predictions = regressor.predict(final_X_test)

In [15]:
submission = pd.read_csv('/content/sample_submission.csv', index_col='id')
display(submission.head())

,target
id,
0,0.5
2,0.5
6,0.5
7,0.5
10,0.5


In [16]:
submission['target'] = predictions
submission.to_csv('xgboost.csv')

In [17]:
!kaggle competitions submit tabular-playground-series-jan-2021 -f xgboost.csv -m "Using XGBoost"

100% 3.07M/3.07M [00:00<00:00, 10.7MB/s]
Successfully submitted to Tabular Playground Series - Jan 2021

In [18]:
submission.head()

,target
id,
0,7.895567
2,7.904824
6,7.935088
7,8.290462
10,8.117891
